In [1]:
import pandas as pd

In [3]:
# Load the hourly data
file_path = '../Studies/oura_feats_bin01_23_hourly.csv'
hourly_data = pd.read_csv(file_path)

# Convert 'hour' to datetime and set it as the index
hourly_data['hour'] = pd.to_datetime(hourly_data['hour'])
hourly_data.set_index('hour', inplace=True)

# Interpolate missing values linearly
hourly_data.interpolate(method='linear', inplace=True)

# Forward fill and backward fill
hourly_data.fillna(method='ffill', inplace=True)
hourly_data.fillna(method='bfill', inplace=True)

# Identify and remove participants or days with excessive missing data
missing_data_threshold = 0.5  # 50% threshold
grouped_data = hourly_data.groupby([hourly_data.index.date, 'participant_id']).apply(
    lambda group: group.isna().mean()
)
excessive_missing_data = grouped_data[grouped_data > missing_data_threshold].dropna(how='all')

# Handle days with excessive missing data (if any)

# Prepare the final datasets
# Dataset with imputed values
# hourly_data.to_csv('/path/to/amed_clean_hour_template.csv')

# Dataset with missing values imputed as zeros
hour_data_zeros = hourly_data.fillna(0)
# hour_data_zeros.to_csv('/path/to/amed_clean_hour_zeros.csv')
